In [20]:
pip install pyspark


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install graphframes

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install findspark


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [23]:
import findspark
findspark.init()

In [26]:
from pyspark import SparkContext,SQLContext
from graphframes import *
import pyspark.sql.functions as f

sc = SparkContext.getOrCreate()
sc.addPyFile('D:/UMKC/PB/Project/spark-2.4.5-bin-hadoop2.7/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar')
sqlcontext=SQLContext(sc)

In [27]:
station_data_df= sqlcontext.read.format("csv").option("header", "true").csv(r'station_data.csv')
trips_df= sqlcontext.read.format("csv").option("header", "true").csv(r'trip_data.csv')


In [28]:
station_data_df.show()

+----------+--------------------+---------+-----------+---------+------------+------------+
|station_id|                name|      lat|       long|dockcount|    landmark|installation|
+----------+--------------------+---------+-----------+---------+------------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|    San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|    San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|    San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|    San Jose|    8/5/2013|
|         6|    San Pedro Square|37.336721|-121.894074|       15|    San Jose|    8/7/2013|
|         7|Paseo de San Antonio|37.333798|-121.886943|       15|    San Jose|    8/7/2013|
|         8| San Salvador at 1st|37.330165|-121.885831|       15|    San Jose|    8/5/2013|
|         9|           Japantown|37.348742|-121.894715|       15|    San Jose|  

In [29]:
station_data_vertices=station_data_df.withColumnRenamed("name","id").select("id").distinct()
station_data_vertices.show()

+--------------------+
|                  id|
+--------------------+
|       2nd at Folsom|
|California Ave Ca...|
|Washington at Kea...|
|Powell at Post (U...|
| Golden Gate at Polk|
|Yerba Buena Cente...|
|   Market at Sansome|
|         MLK Library|
|     Spear at Folsom|
|           Japantown|
|Commercial at Mon...|
|Paseo de San Antonio|
| San Salvador at 1st|
|Rengstorff Avenue...|
|     Townsend at 7th|
|Civic Center BART...|
|         Ryland Park|
|San Jose Diridon ...|
|San Jose Civic Ce...|
|     Post at Kearney|
+--------------------+
only showing top 20 rows



In [30]:
trip_edges=trips_df.withColumnRenamed("Start Station","src")\
.withColumnRenamed("End Station","dst")\
.select("src","dst")

In [31]:
trip_edges.show()

+--------------------+--------------------+
|                 src|                 dst|
+--------------------+--------------------+
|Harry Bridges Pla...|San Francisco Cal...|
|San Antonio Shopp...|Mountain View Cit...|
|      Post at Kearny|   2nd at South Park|
|  San Jose City Hall| San Salvador at 1st|
|Embarcadero at Fo...|Embarcadero at Sa...|
|Yerba Buena Cente...|San Francisco Cal...|
|Embarcadero at Fo...|Embarcadero at Sa...|
|Embarcadero at Sa...|   Steuart at Market|
|     Beale at Market|Temporary Transba...|
|      Post at Kearny|South Van Ness at...|
|Embarcadero at Sa...|   Market at Sansome|
|      Market at 10th|San Francisco Cal...|
|       Market at 4th|Grant Avenue at C...|
|   Market at Sansome|Broadway St at Ba...|
|Temporary Transba...|San Francisco Cal...|
|San Francisco Cal...|     Townsend at 7th|
|University and Em...|Cowper at University|
|     Spear at Folsom|San Francisco Cal...|
|Temporary Transba...|Grant Avenue at C...|
|San Francisco Cal...|Broadway S

In [32]:
g=GraphFrame(station_data_vertices,trip_edges)
g

GraphFrame(v:[id: string], e:[src: string, dst: string])

In [33]:
g.triangleCount().show()

+-----+--------------------+
|count|                  id|
+-----+--------------------+
|  496|       2nd at Folsom|
|   23|California Ave Ca...|
|    0|Washington at Kea...|
|  496|Powell at Post (U...|
|  496| Golden Gate at Polk|
|  496|Yerba Buena Cente...|
|  496|   Market at Sansome|
|   90|         MLK Library|
|  496|     Spear at Folsom|
|   77|           Japantown|
|  496|Commercial at Mon...|
|   81|Paseo de San Antonio|
|   23|Rengstorff Avenue...|
|   61| San Salvador at 1st|
|  496|     Townsend at 7th|
|  496|Civic Center BART...|
|   41|         Ryland Park|
|   90|San Jose Diridon ...|
|   63|San Jose Civic Ce...|
|    0|     Post at Kearney|
+-----+--------------------+
only showing top 20 rows



In [34]:
g.shortestPaths(landmarks=["2nd at Folsom", "Japantown"]).show()

+--------------------+--------------------+
|                  id|           distances|
+--------------------+--------------------+
|       2nd at Folsom|[2nd at Folsom -> 0]|
|      Market at 10th|[2nd at Folsom -> 1]|
|California Ave Ca...|                  []|
|Washington at Kea...|                  []|
|Redwood City Publ...|                  []|
|Powell at Post (U...|[2nd at Folsom -> 1]|
| Golden Gate at Polk|[2nd at Folsom -> 2]|
|    Adobe on Almaden|    [Japantown -> 2]|
|Broadway St at Ba...|[2nd at Folsom -> 1]|
|Yerba Buena Cente...|[2nd at Folsom -> 1]|
|     Beale at Market|[2nd at Folsom -> 1]|
|   Market at Sansome|[2nd at Folsom -> 1]|
|         MLK Library|    [Japantown -> 1]|
|     Spear at Folsom|[2nd at Folsom -> 1]|
|       5th at Howard|[2nd at Folsom -> 1]|
|           Japantown|    [Japantown -> 0]|
|Commercial at Mon...|[2nd at Folsom -> 1]|
|    San Pedro Square|    [Japantown -> 1]|
|Paseo de San Antonio|    [Japantown -> 1]|
|Redwood City Medi...|          

In [35]:
result = g.pageRank(resetProbability=0.15, tol=0.01)
result.vertices.show()
result.edges.show()

+--------------------+-------------------+
|                  id|           pagerank|
+--------------------+-------------------+
|       2nd at Folsom| 0.6019976457886556|
|      Market at 10th|  1.052787200094611|
|California Ave Ca...| 0.9108234167101502|
|Washington at Kea...| 0.1688962693649758|
|Redwood City Publ...| 0.3772310498004255|
|Powell at Post (U...| 0.5155331272749033|
| Golden Gate at Polk|0.46210834153699853|
|    Adobe on Almaden| 0.5363219791463251|
|Broadway St at Ba...| 0.6542773822104675|
|Yerba Buena Cente...|    0.6446074415843|
|     Beale at Market| 0.7386610420274149|
|   Market at Sansome| 1.3054243296675345|
|         MLK Library| 0.7773824929442179|
|     Spear at Folsom| 0.6982859901236311|
|       5th at Howard| 0.8188175867235038|
|           Japantown| 0.9592388779037915|
|Commercial at Mon...| 0.6637610436731323|
|    San Pedro Square| 1.8004572705485629|
|Paseo de San Antonio| 1.2407143489672836|
|Redwood City Medi...| 0.5246180540767911|
+----------

In [15]:
g.vertices.write.parquet('vertices')
g.edges.write.parquet('edges')

In [16]:
result = g.labelPropagation(maxIter=5)
result.select("id", "label").show()

+--------------------+-------------+
|                  id|        label|
+--------------------+-------------+
|San Jose Diridon ...|1365799600128|
|Embarcadero at Va...|1657857376256|
|    San Pedro Square| 292057776128|
|Arena Green / SAP...|1365799600128|
|San Francisco Cal...|1657857376256|
|Washington at Kea...|  17179869184|
|San Francisco Cal...|1657857376256|
|   Steuart at Market|1657857376256|
|Broadway St at Ba...|1657857376256|
|         Ryland Park| 292057776128|
|   Market at Sansome|1657857376256|
|    Davis at Jackson|1657857376256|
|San Mateo County ...| 730144440320|
|Redwood City Calt...|1151051235328|
|         MLK Library| 292057776128|
|   Franklin at Maple| 730144440320|
|Mountain View Cal...| 463856467969|
|Grant Avenue at C...|1657857376256|
| Golden Gate at Polk|1657857376256|
|San Antonio Shopp...| 661424963584|
+--------------------+-------------+
only showing top 20 rows



In [17]:
paths = g.bfs("id = 'Japantown'", "id = 'MLK Library'")
paths.show()

+-----------+--------------------+-------------+
|       from|                  e0|           to|
+-----------+--------------------+-------------+
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
+-----------+--------------------+-------------+

